<a href="https://colab.research.google.com/github/JustFineNeuro/LDS_CONTROL_HMM/blob/main/CreatingLDSANDCONTROLLERs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy import signal
import numpy as np

Specify a linear state space model of the form w/signal independent noise:

Evolution/process equation
$x(t+1)=Ax(t)+Bu(t)+Normal(0,\sigma_1)$

Observation equation
$y(t)=Cx(t)+Dw(t)+Normal(0,\sigma_2)$


References
https://www.textroad.com/pdf/JAEBS/J.%20Appl.%20Environ.%20Biol.%20Sci.,%205(4S)1-11,%202015.pdf

In [ ]:
#Sampling frequency =1/60 Hz

dt=1/60

#Discretized system for constant velocity joystick: 2D
A=np.matrix([[1 ,dt,0,0],[0 ,0,0,0],[0 ,0,1,dt],[0 ,0,0,0]])
B=np.array([[0],[1*dt],[0],[1*dt]])

C=np.array([[1],[1]])


[[0.        ]
 [0.01666667]]


In [ ]:
sys=sys.to_discrete(dt)

  def dynamics_closedloop(self, D: float, B: float, L: np.ndarray):

    s = np.zeros(self.T)  # states initialization
    s[0] = self.ini_state
    noise = np.random.normal(0, self.noise_var, self.T)
    a = np.zeros(self.T - 1)

    for t in range(self.T - 1):
      # calculate the current action
      a[t] = L[t] * s[t]
      # calculate the next state
      s[t + 1] = D * s[t] + B * a[t] + noise[t]

    return s, a


In [ ]:

A*x

matrix([[1.03333333],
        [0.        ]])